In [ ]:
# Install YOLOv8 and DeepSORT dependencies
!pip install ultralytics
!git clone https://github.com/ZQPei/deep_sort_pytorch.git
%cd deep_sort_pytorch
!pip install -r requirements.txt

# Install additional dependencies
!pip install numpy
!pip install pyyaml


fatal: destination path 'deep_sort_pytorch' already exists and is not an empty directory.
/content/deep_sort_pytorch
ERROR: Invalid requirement: 'torch~=1.12.1+cu113' (from line 37 of requirements.txt)


In [ ]:
# Install Torch and Torchvision
!pip install torch torchvision

In [ ]:
# Install OpenCV and Hydra
!pip install opencv-python
!pip install hydra-core --upgrade

In [ ]:
import hydra
import torch
import math
import cv2
import numpy as np
from pathlib import Path
from collections import deque
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort
from ultralytics import YOLO
from ultralytics.yolo.utils.checks import check_imgsz
from ultralytics.yolo.utils.plotting import Annotator, colors

# Global variables
palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)
data_deque = {}
deepsort = None
object_counter = {}
object_counter1 = {}
line = [(100, 500), (1050, 500)]
speed_line_queue = {}
event_log = []

# Utility Functions
def estimatespeed(Location1, Location2):
    d_pixel = math.sqrt((Location2[0] - Location1[0]) ** 2 + (Location2[1] - Location1[1]) ** 2)
    ppm = 8
    d_meters = d_pixel / ppm
    time_constant = 15 * 3.6
    speed = d_meters * time_constant
    return int(speed)

def init_tracker():
    global deepsort
    cfg_deep = get_config()
    cfg_deep.merge_from_file("deep_sort_pytorch/configs/deep_sort.yaml")
    deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT, max_dist=cfg_deep.DEEPSORT.MAX_DIST,
                        min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE, nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE, max_age=cfg_deep.DEEPSORT.MAX_AGE,
                        n_init=cfg_deep.DEEPSORT.N_INIT, nn_budget=cfg_deep.DEEPSORT.NN_BUDGET, use_cuda=True)

def xyxy_to_xywh(*xyxy):
    bbox_left = min(xyxy[0].item(), xyxy[2].item())
    bbox_top = min(xyxy[1].item(), xyxy[3].item())
    bbox_w = abs(xyxy[0].item() - xyxy[2].item())
    bbox_h = abs(xyxy[1].item() - xyxy[3].item())
    x_c = bbox_left + bbox_w / 2
    y_c = bbox_top + bbox_h / 2
    return x_c, y_c, bbox_w, bbox_h

def compute_color_for_labels(label):
    if label == 0:
        color = (85, 45, 255)
    elif label == 2:
        color = (222, 82, 175)
    elif label == 3:
        color = (0, 204, 255)
    elif label == 5:
        color = (0, 149, 255)
    else:
        color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def get_direction(point1, point2):
    direction_str = ""
    if point1[1] > point2[1]:
        direction_str += "South"
    elif point1[1] < point2[1]:
        direction_str += "North"
    if point1[0] > point2[0]:
        direction_str += "East"
    elif point1[0] < point2[0]:
        direction_str += "West"
    return direction_str

def get_quadrant(center, width, height):
    if center[0] < width / 2 and center[1] < height / 2:
        return 1
    elif center[0] >= width / 2 and center[1] < height / 2:
        return 2
    elif center[0] < width / 2 and center[1] >= height / 2:
        return 3
    else:
        return 4

def log_event(timestamp, quadrant, ball_color, event_type):
    event_log.append(f"{timestamp}, {quadrant}, {ball_color}, {event_type}")

def draw_boxes(img, bbox, names, object_id, identities=None, offset=(0, 0)):
    cv2.line(img, line[0], line[1], (46, 162, 112), 3)
    height, width, _ = img.shape

    for key in list(data_deque):
        if key not in identities:
            data_deque.pop(key)

    for i, box in enumerate(bbox):
        x1, y1, x2, y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[1]
        y2 += offset[1]

        center = (int((x2 + x1) / 2), int((y2 + y1) / 2))
        quadrant = get_quadrant(center, width, height)

        id = int(identities[i]) if identities is not None else 0

        if id not in data_deque:
            data_deque[id] = deque(maxlen=64)
            speed_line_queue[id] = []

        color = compute_color_for_labels(object_id[i])
        obj_name = names[object_id[i]]
        label = f'{id}:{obj_name}'

        data_deque[id].appendleft(center)
        if len(data_deque[id]) >= 2:
            direction = get_direction(data_deque[id][0], data_deque[id][1])
            object_speed = estimatespeed(data_deque[id][1], data_deque[id][0])
            speed_line_queue[id].append(object_speed)
            if intersect(data_deque[id][0], data_deque[id][1], line[0], line[1]):
                cv2.line(img, line[0], line[1], (255, 255, 255), 3)
                if "South" in direction:
                    if obj_name not in object_counter:
                        object_counter[obj_name] = 1
                    else:
                        object_counter[obj_name] += 1
                if "North" in direction:
                    if obj_name not in object_counter1:
                        object_counter1[obj_name] = 1
                    else:
                        object_counter1[obj_name] += 1

            timestamp = int(cv2.getTickCount() / cv2.getTickFrequency())
            if direction:
                log_event(timestamp, quadrant, color, direction)

    return img


class DetectionPredictor:
    def __init__(self, model_path='yolov8n.pt'):
        self.model = YOLO(model_path)  # Load the model

    def get_annotator(self, img):
        return Annotator(img)

    def preprocess(self, img):
        img = torch.from_numpy(img).to(self.model.device)
        img = img.half() if self.model.fp16 else img.float()
        img /= 255
        return img

    def postprocess(self, preds, img, orig_img):
        preds = ops.non_max_suppression(preds, conf_thres=0.25, iou_thres=0.45)
        for i, pred in enumerate(preds):
            shape = orig_img[i].shape if isinstance(orig_img, list) else orig_img.shape
            pred[:, :4] = ops.scale_boxes(img.shape[2:], pred[:, :4], shape).round()
        return preds

    def write_results(self, preds, img, orig_img):
        all_outputs = []
        log_string = ""
        self.seen += 1
        im0 = orig_img.copy()
        det = preds[0]

        if len(det) == 0:
            return log_string

        for c in det[:, 5].unique():
            n = (det[:, 5] == c).sum()
            log_string += f"{n} {self.model.names[int(c)]}{'s' * (n > 1)}, "

        xywh_bboxs = []
        confs = []
        oids = []
        outputs = []
        for *xyxy, conf, cls in reversed(det):
            x_c, y_c, bbox_w, bbox_h = xyxy_to_xywh(*xyxy)
            xywh_obj = [x_c, y_c, bbox_w, bbox_h]
            xywh_bboxs.append(xywh_obj)
            confs.append([conf.item()])
            oids.append(int(cls))

        xywhs = torch.Tensor(xywh_bboxs)
        confss = torch.Tensor(confs)

        outputs = deepsort.update(xywhs, confss, oids, im0)
        if len(outputs) > 0:
            bbox_xyxy = outputs[:, :4]
            identities = outputs[:, -2]
            object_id = outputs[:, -1]

            draw_boxes(im0, bbox_xyxy, self.model.names, object_id, identities)
        return log_string


@hydra.main(version_base=None, config_path=str(Path(__file__).parent), config_name="config.yaml")
def predict(cfg):
    init_tracker()
    cfg.model = cfg.model or "yolov8n.pt"
    cfg.imgsz = check_imgsz(cfg.imgsz, min_dim=2)
    cfg.source = "AI Assignment video.mp4"  # Replace with the path to your video file
    predictor = DetectionPredictor(cfg.model)
    predictor()

    with open("event_log.txt", "w") as file:
        file.write("Time, Quadrant Number, Ball Colour, Type (Entry or Exit)\n")
        for event in event_log:
            file.write(event + "\n")


if __name__ == "__main__":
    predict()


ModuleNotFoundError: No module named 'ultralytics.yolo'

In [ ]:
from ultralytics.yolo.utils.checks import check_imgsz
from ultralytics.yolo.utils.plotting import Annotator, colors

# Check if the modules are imported correctly
print("Modules imported successfully.")

ModuleNotFoundError: No module named 'ultralytics.yolo'